In [ ]:
#Import the Libraries and CSV File
%matplotlib inline
import pandas as pd #import pandas
import re #import regex
path= ('/US_National_Parks/df_2.csv')
df = pd.read_csv(path)
df.head(2)

,Unnamed: 0,Name,Image,Location,Date established as park[7][12],Area (2021)[13],Recreation visitors (2021)[11],Description
0,0,Acadia,NaN,"Maine.mw-parser-output .geo-default,.mw-parser...","February 26, 1919","49,071.40 acres (198.6 km2)",4069098,Covering most of Mount Desert Island and other...
1,1,American Samoa,NaN,American Samoa14°15′S 170°41′W﻿ / ﻿14.25°S 170...,"October 31, 1988","8,256.67 acres (33.4 km2)",8495,The southernmost national park is on three Sam...


In [ ]:
#Drop unwanted columns
dropColumns = ['Unnamed: 0', 'Image',]
df = df.drop(columns=dropColumns)

In [ ]:
# Rename specific columns
columnNames = {'Date established as park[7][12]': 'Date Established', 'Area (2021)[13]': 'Area', 'Recreation visitors (2021)[11]': 'Recreation Visitors'}
df = df.rename(columns=columnNames)
df.head(1)

,Name,Location,Date Established,Area,Recreation Visitors,Description
0,Acadia,"Maine.mw-parser-output .geo-default,.mw-parser...","February 26, 1919","49,071.40 acres (198.6 km2)",4069098,Covering most of Mount Desert Island and other...


In [ ]:
#Lets extract the Cordinates details from the 'Location' column

#regex pattern to extract coordinates
pattern = r"(\d+°\d+′[NS] \d+°\d+′[EW]﻿ / ﻿\d+\.\d+°[NS] \d+\.\d+°[EW])"
# Extract coordinates from the 'Location' column
df['Coordinates'] = df['Location'].str.extract(pattern)
df.head(1)

,Name,Location,Date Established,Area,Recreation Visitors,Description,Coordinates
0,Acadia,"Maine.mw-parser-output .geo-default,.mw-parser...","February 26, 1919","49,071.40 acres (198.6 km2)",4069098,Covering most of Mount Desert Island and other...,44°21′N 68°13′W﻿ / ﻿44.35°N 68.21°W


In [ ]:
# Lets extract the State name from the 'Location' column
pattern2 = r"([A-Za-z\s]+)"
df['State'] = df['Location'].str.extract(pattern2)
df.head(1)

,Name,Location,Date Established,Area,Recreation Visitors,Description,Coordinates,State
0,Acadia,"Maine.mw-parser-output .geo-default,.mw-parser...","February 26, 1919","49,071.40 acres (198.6 km2)",4069098,Covering most of Mount Desert Island and other...,44°21′N 68°13′W﻿ / ﻿44.35°N 68.21°W,Maine


In [ ]:
#Lets Check for null values and fix it
null_rows = df[df['Coordinates'].isnull()]
print(null_rows)

             Name                                           Location  \
32  Indiana Dunes  Indiana41°39′12″N 87°03′09″W﻿ / ﻿41.6533°N 87....   

     Date Established                        Area  Recreation Visitors  \
32  February 15, 2019  15,349.08 acres (62.1 km2)              3177210   

                                          Description Coordinates    State  
32  Previously designated a national lakeshore, pa...         NaN  Indiana  


In [ ]:
df[df['Name'] == "Indiana Dunes"]['Location']

32    Indiana41°39′12″N 87°03′09″W﻿ / ﻿41.6533°N 87....
Name: Location, dtype: object

In [ ]:
# Update the value in 'Coordinates' column for the row with 'Name' == "Indiana Dunes"
df.loc[df['Name'] == "Indiana Dunes", 'Coordinates'] = "41°39′12″N 87°03′09″W﻿ / ﻿41.6533°N 87.0524°W"

In [ ]:
df['Coordinates'].isnull().sum()

0

In [ ]:
# Split the values in the 'Coordinates' column based on the '/' space
df[['Coordinate1', 'Coordinate2']] = df['Coordinates'].str.split(' / ', expand=True)
df.head(2)

,Name,Location,Date Established,Area,Recreation Visitors,Description,Coordinates,State,Coordinate1,Coordinate2
0,Acadia,"Maine.mw-parser-output .geo-default,.mw-parser...","February 26, 1919","49,071.40 acres (198.6 km2)",4069098,Covering most of Mount Desert Island and other...,44°21′N 68°13′W﻿ / ﻿44.35°N 68.21°W,Maine,44°21′N 68°13′W﻿,﻿44.35°N 68.21°W
1,American Samoa,American Samoa14°15′S 170°41′W﻿ / ﻿14.25°S 170...,"October 31, 1988","8,256.67 acres (33.4 km2)",8495,The southernmost national park is on three Sam...,14°15′S 170°41′W﻿ / ﻿14.25°S 170.68°W,American Samoa,14°15′S 170°41′W﻿,﻿14.25°S 170.68°W


In [ ]:
#Lets extract the Latitude and Longitude values for Plotting the map
df['Latitude'] = df['Coordinate2'].str.extract(r'([0-9.]+)°[NSEW]')
df['Longitude'] = df['Coordinate2'].str.extract(r'[NSEW]\s*([0-9.]+)°[NSEW]')

# Remove leading/trailing whitespace or special characters
df['Latitude'] = df['Latitude'].str.strip()
df['Longitude'] = df['Longitude'].str.strip()

# Convert latitude and longitude to numeric values
df['Latitude'] = pd.to_numeric(df['Latitude'])
df['Longitude'] = pd.to_numeric(df['Longitude'])

# Adjust latitude and longitude based on direction (N/S or E/W)
df.loc[df['Coordinate2'].str.contains('S'), 'Latitude'] *= -1
df.loc[df['Coordinate2'].str.contains('W'), 'Longitude'] *= -1

df.head(2)

,Name,Location,Date Established,Area,Recreation Visitors,Description,Coordinates,State,Coordinate1,Coordinate2,Latitude,Longitude
0,Acadia,"Maine.mw-parser-output .geo-default,.mw-parser...","February 26, 1919","49,071.40 acres (198.6 km2)",4069098,Covering most of Mount Desert Island and other...,44°21′N 68°13′W﻿ / ﻿44.35°N 68.21°W,Maine,44°21′N 68°13′W﻿,﻿44.35°N 68.21°W,44.35,-68.21
1,American Samoa,American Samoa14°15′S 170°41′W﻿ / ﻿14.25°S 170...,"October 31, 1988","8,256.67 acres (33.4 km2)",8495,The southernmost national park is on three Sam...,14°15′S 170°41′W﻿ / ﻿14.25°S 170.68°W,American Samoa,14°15′S 170°41′W﻿,﻿14.25°S 170.68°W,-14.25,-170.68


In [ ]:
# Extract the date and remove excess characters
df['Date Established'] = df['Date Established'].str.replace(r'\[\d+\]', '')

<ipython-input-14-23a44ceb594c>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Date Established'] = df['Date Established'].str.replace(r'\[\d+\]', '')


In [ ]:
df['Date_Established_1'] = pd.to_datetime(df['Date Established'], errors='coerce')
df['Year Established'] = df['Date_Established_1'].dt.year

In [ ]:
df.head(1)

,Name,Location,Date Established,Area,Recreation Visitors,Description,Coordinates,State,Coordinate1,Coordinate2,Latitude,Longitude,Date_Established_1,Year Established
0,Acadia,"Maine.mw-parser-output .geo-default,.mw-parser...","February 26, 1919","49,071.40 acres (198.6 km2)",4069098,Covering most of Mount Desert Island and other...,44°21′N 68°13′W﻿ / ﻿44.35°N 68.21°W,Maine,44°21′N 68°13′W﻿,﻿44.35°N 68.21°W,44.35,-68.21,1919-02-26,1919


In [ ]:
#Now lets get the Acres data for Area and eliminate km2 info
df['Acres'] = df['Area'].str.split(' ', expand=True)[0].str.replace(',', '')
df.head(1)

,Name,Location,Date Established,Area,Recreation Visitors,Description,Coordinates,State,Coordinate1,Coordinate2,Latitude,Longitude,Date_Established_1,Year Established,Acres
0,Acadia,"Maine.mw-parser-output .geo-default,.mw-parser...","February 26, 1919","49,071.40 acres (198.6 km2)",4069098,Covering most of Mount Desert Island and other...,44°21′N 68°13′W﻿ / ﻿44.35°N 68.21°W,Maine,44°21′N 68°13′W﻿,﻿44.35°N 68.21°W,44.35,-68.21,1919-02-26,1919,49071.40


In [ ]:
df['State'] = df['State'].replace('U', 'U.S. Virgin Islands')
df['State'].unique()

array(['Maine', 'American Samoa', 'Utah', 'South Dakota', 'Texas',
       'Florida', 'Colorado', 'New Mexico', 'California',
       'South Carolina', 'Oregon', 'Ohio', 'Alaska', 'Missouri',
       'Montana', 'Arizona', 'Wyoming', 'Nevada', 'North Carolina',
       'Hawaii', 'Arkansas', 'Indiana', 'Michigan', 'Kentucky',
       'Washington', 'West Virginia', 'Virginia', 'North Dakota',
       'U.S. Virgin Islands', 'Minnesota'], dtype=object)

In [ ]:
states = {
    'AK': 'Alaska',
    'AL': 'Alabama',
    'AR': 'Arkansas',
    'AZ': 'Arizona',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DC': 'District of Columbia',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'IA': 'Iowa',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'MA': 'Massachusetts',
    'MD': 'Maryland',
    'ME': 'Maine',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MO': 'Missouri',
    'MS': 'Mississippi',
    'MT': 'Montana',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'NE': 'Nebraska',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NV': 'Nevada',
    'NY': 'New York',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VA': 'Virginia',
    'VT': 'Vermont',
    'WA': 'Washington',
    'WI': 'Wisconsin',
    'WV': 'West Virginia',
    'WY': 'Wyoming',
    'AS': 'American Samoa',
    'VI': 'U.S. Virgin Islands',
}

In [ ]:
df['State Code'] = df['State'].map({v: k for k, v in states.items()})

In [ ]:
#df.info()

In [ ]:
#Print null rows
#null_rows = df[df['State Code'].isnull()]
#print(null_rows)

In [ ]:
# Convert 'Acres' column to numeric data type
df['Acres'] = pd.to_numeric(df['Acres'], errors='coerce')

In [ ]:
#Lets Drop unwanted columns again
dropColumns = ['Location', 'Coordinates','Coordinate1',]
df = df.drop(columns=dropColumns)

In [ ]:
df.head()

,Name,Date Established,Area,Recreation Visitors,Description,State,Coordinate2,Latitude,Longitude,Date_Established_1,Year Established,Acres,State Code
0,Acadia,"February 26, 1919","49,071.40 acres (198.6 km2)",4069098,Covering most of Mount Desert Island and other...,Maine,﻿44.35°N 68.21°W,44.35,-68.21,1919-02-26,1919,49071.40,ME
1,American Samoa,"October 31, 1988","8,256.67 acres (33.4 km2)",8495,The southernmost national park is on three Sam...,American Samoa,﻿14.25°S 170.68°W,-14.25,-170.68,1988-10-31,1988,8256.67,AS
2,Arches,"November 12, 1971","76,678.98 acres (310.3 km2)",1806865,"This site features more than 2,000 natural san...",Utah,﻿38.68°N 109.57°W,38.68,-109.57,1971-11-12,1971,76678.98,UT
3,Badlands,"November 10, 1978","242,755.94 acres (982.4 km2)",1224226,"The Badlands are a collection of buttes, pinna...",South Dakota,﻿43.75°N 102.50°W,43.75,-102.50,1978-11-10,1978,242755.94,SD
4,Big Bend,"June 12, 1944","801,163.21 acres (3,242.2 km2)",581220,Named for the prominent bend in the Rio Grande...,Texas,﻿29.25°N 103.25°W,29.25,-103.25,1944-06-12,1944,801163.21,TX


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Name                 63 non-null     object        
 1   Date Established     63 non-null     object        
 2   Area                 63 non-null     object        
 3   Recreation Visitors  63 non-null     int64         
 4   Description          63 non-null     object        
 5   State                63 non-null     object        
 6   Coordinate2          63 non-null     object        
 7   Latitude             63 non-null     float64       
 8   Longitude            63 non-null     float64       
 9   Date_Established_1   63 non-null     datetime64[ns]
 10  Year Established     63 non-null     int64         
 11  Acres                63 non-null     float64       
 12  State Code           63 non-null     object        
dtypes: datetime64[ns](1), float64(3), int

In [ ]:
df.to_csv("/content/drive/MyDrive/Colab Notebooks/PracticeStatistics_GitHub/US_National_Parks/USNationalPark_Dataset_Cleaned.csv")